In [1]:
#Global variables
import socket
import os
ENV_HOSTNAME = socket.gethostname()
print('ENV_HOSTNAME:' + ENV_HOSTNAME)

#store defaults for Jacob here:
ENV_FOLDER_DATA = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data\\'
ENV_FOLDER_DATA_PROC = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data_Proc\\'
ENV_PRATT = ''


if ENV_HOSTNAME == 'JWGamingPC':
    ENV_FOLDER_DATA = 'E:\\W4732 Computer Vision\\Final Paper Data\\'
    ENV_FOLDER_DATA_PROC = 'E:\\W4732 Computer Vision\\Final Paper Data Proc\\'
    ENV_PRATT = ''

ENV_HOSTNAME:The-Beast


In [2]:
#Global variables
import cv2
import numpy as np
import matplotlib.pyplot as plt

def preprocess_video(video_path, desired_fps=10, target_resolution=(640, 360)):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = cv2.resize(frame, target_resolution)
        frames.append(frame)

    cap.release()

    frame_interval = int(10 / desired_fps)
    sampled_frames = frames[::frame_interval]

    video_data = np.array(sampled_frames)
    video_data = video_data / 255.0

    
    # video_data = np.expand_dims(video_data, axis=-1)

    if video_data.shape[0] < 40:
        # Pad the video data with zeros to make it at least 40 frames long
        padding = np.zeros((40 - video_data.shape[0], video_data.shape[1], video_data.shape[2]))
        video_data = np.concatenate((video_data, padding), axis=0)
    elif video_data.shape[0] > 40:
        # Trim the video data to be exactly 40 frames long
        video_data = video_data[:40]
    
    return video_data

# Example usage:
# video_path = ENV_FOLDER_DATA_PROC + 'clips\\501\\501_524_528.mp4'
# preprocessed_data = preprocess_video(video_path)
# print("Preprocessed video data shape:", preprocessed_data.shape)

# video_dir = ENV_FOLDER_DATA_PROC + 'clips\\595\\'

# for video in os.listdir(video_dir):
#     video_path = video_dir + video
#     preprocessed_data = preprocess_video(video_path)
#     display_frame = np.vstack(preprocessed_data)

#     plt.imshow(display_frame, cmap='gray')
#     plt.axis('off')
#     plt.show()
#     # # display frames
#     # for frame in preprocessed_data:
#     #     cv2.imshow('frame', frame)
#     # break
#     #print("Preprocessed video data shape:", preprocessed_data.shape)

In [3]:
# Create the Data loader

import torch
from torch.utils.data import Dataset
import cv2
import numpy as np
import pandas as pd

class VideoDataset(Dataset):
    def __init__(self, video_paths, desired_fps=10, target_resolution=(640, 360), label_type='p'):
        self.video_paths = video_paths
        self.desired_fps = desired_fps
        self.target_resolution = target_resolution
        self.label_type = label_type

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        video_data = preprocess_video(video_path, self.desired_fps, self.target_resolution)
        video_data = np.expand_dims(video_data, axis=0)

        features = torch.tensor(video_data, dtype=torch.float32)

        episode_number = video_path.split('\\')[-2]
        seg_number = video_path.split('_seg')[-1].split('.')[0]
        pickle_path = ENV_FOLDER_DATA_PROC + 'targetdf\\' + episode_number + '.pickle'
        target_df = pd.read_pickle(pickle_path)

        #find row where seg number is equal to seg number and type is equal to label type
        target_row = target_df[(target_df['seg'] == int(seg_number)) & (target_df['type'] == self.label_type)]

        #extract the data from columns 3:43

        audio_data = target_row.iloc[:, 4:45].values

        labels = torch.tensor(audio_data.astype(float).flatten())
        
        labels = torch.nan_to_num(labels)

        if self.label_type == 'p':
            labels = labels / 600.0
        elif self.label_type == 'i':
            labels = labels / 100.0
        elif self.label_type == 's':
            labels = labels / 2.0
        elif self.label_type == 'h':
            labels = (labels+40.0) /100.0
    
        
        # if len(features) != 40 or len(labels) != 41:
        #     #print(episode_number, seg_number, len(features), len(labels))
        #     #write to ENV_FOLDER_DATA_PROC + 'invalid_vid_paths.txt'
        #     with open(ENV_FOLDER_DATA_PROC + 'invalid_vid_paths.txt', 'a') as f:
        #         f.write(video_path + '\n')
            
        #     labels = torch.zeros(41)

        return features, labels


from torch.utils.data import DataLoader

clips_dir = ENV_FOLDER_DATA_PROC + 'clips\\'
# Example usage
video_paths = []


for root, dirs, files in os.walk(clips_dir):
    for file in files:
        if file.endswith(".mp4"):
            video_paths.append(os.path.join(root, file))


training_paths = video_paths[:int(len(video_paths) * 0.8)]
test_paths = video_paths[int(len(video_paths) * 0.8):]

training_dataset = VideoDataset(training_paths)
test_dataset = VideoDataset(test_paths)

training_dataloader = DataLoader(training_dataset, batch_size=12, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True)


# dataset = VideoDataset(video_paths)
# # discard all data points that are None,None
# #dataset = [data for data in dataset if data != (None, None)]
# dataloader = DataLoader(dataset, batch_size=128, shuffle=True)


# print(len(training_dataset))
# # Iterate over batches
# for batch_idx, (data, labels) in enumerate(training_dataloader):
#     # 'data' will have shape (batch_size, 1, 40, 640, 360)
#     # We can extract the first frame (index 0 along the frame dimension)
#     first_frame_batch = data[:, 0, 0]

#     # Iterate through each sample in the batch
#     for i in range(len(first_frame_batch)):
#         # Get the first frame of the sample and its corresponding label
#         first_frame = first_frame_batch[i]
#         label = labels[i]

#         # Convert the torch tensor to a numpy array and transpose dimensions
#         # Assuming the tensor is in (channel, depth, height, width) format
#         first_frame_np = first_frame.numpy()

#         # Display the first frame using matplotlib
#         plt.figure(figsize=(8, 6))
#         plt.imshow(first_frame_np.squeeze(), cmap='gray')  # Squeeze out the channel dimension
#         plt.title(f'Batch {batch_idx + 1}, Sample {i + 1}, Label: {label}')
#         plt.axis('off')
#         plt.show()

#     # Only display the first batch
#     break



In [4]:
import os
import shutil

def move_files_to_folder(file_list_path, destination_folder):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Read the list of file paths from the text file
    with open(file_list_path, 'r') as file:
        file_paths = file.readlines()

    # Remove leading/trailing whitespaces and newline characters
    file_paths = [path.strip() for path in file_paths]

    # Move each file to the destination folder
    for file_path in file_paths:
        if os.path.exists(file_path):  # Check if file exists
            file_name = os.path.basename(file_path)
            destination_path = os.path.join(destination_folder, file_name)
            shutil.move(file_path, destination_path)
            print(f"Moved '{file_path}' to '{destination_path}'")
        else:
            print(f"File '{file_path}' not found.")

# Example usage:
# file_list_path = ENV_FOLDER_DATA_PROC + 'invalid_vid_paths.txt'  # Path to the text file containing the list of file paths
# destination_folder = ENV_FOLDER_DATA_PROC + 'invalid_clips\\'  # Destination folder to move the files to
# move_files_to_folder(file_list_path, destination_folder)

In [5]:
#Code to test hypotheses

# 2 Layers of 3D CONV
# 3 Layers of 2D CONV
# Max pooling after each layer
# Batch normalization after each layer
# Flatten
# 3 Fully connected layers

# Leaky ReLU activation function

# adam optimizer
# Loss Function: Cross Entropy

import torch.nn as nn
import torch.nn.functional as F


class VideoConvNet(nn.Module):
    def __init__(self, in_channels=1, num_classes=41):
        super().__init__()
        # TODO: Initialize network layers
        
        self.layer1 = nn.Sequential(
            nn.Conv3d(in_channels=in_channels, out_channels=8, kernel_size=6, padding=3, stride=3),
            nn.BatchNorm3d(8),
            nn.LeakyReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv3d(in_channels=8, out_channels=32, kernel_size=5, padding=3, stride=3),
            nn.BatchNorm3d(32),
            nn.LeakyReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv3d(in_channels=32, out_channels=64, kernel_size=4, padding=3, stride=2),
            nn.BatchNorm3d(64),
            nn.LeakyReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2))
        # self.layer4 = nn.Sequential(
        #     nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
        #     nn.BatchNorm2d(128),
        #     nn.LeakyReLU(),
        #     nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Sequential(
            nn.Linear(960, 256),
            nn.LeakyReLU(),
            nn.Dropout(0.5))
        self.fc2 = nn.Sequential(
            nn.Linear(256, 128),
            nn.LeakyReLU(),
            nn.Dropout(0.5))
        self.fc3 = nn.Sequential(
            nn.Linear(128, num_classes))
        







    def forward(self, x):
        # TODO: Implement the forward pass with using the layers defined above
        #       and the proper activation functions
        #print("input: ", x.shape)
        x = self.layer1(x)
        #print("layer1: ", x.shape)
        x = self.layer2(x)
        #print("layer2: ", x.shape) 
        x = self.layer3(x)
        #print("layer3: ", x.shape)
        #x = self.layer4(x)
        #print("layer4: ", x.shape)
        x = x.view(x.size(0), -1)
        #print("flatten: ", x.shape)
        x = self.fc1(x)
        #print("fc1: ", x.shape)
        x = self.fc2(x)
        #print("fc2: ", x.shape)
        x = self.fc3(x)
        #print("fc3: ", x.shape)
        return x

vidNet = VideoConvNet()





In [6]:
# # Assuming you have a CUDA-enabled GPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Move the model to the GPU
# vidNet.to(device)

# # Loss function and optimizer
# import torch.optim as optim
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(vidNet.parameters(), lr=0.01)

# # Training the network
# num_epochs = 10

# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for i, data in enumerate(training_dataloader, 0):
#         inputs, labels = data
#         inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU

#         optimizer.zero_grad()

#         outputs = vidNet(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         if i % 10 == 9:
#             print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 10}")
#             running_loss = 0.0



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vidNet.parameters(), lr=0.001)
def train_network(net, n_epochs=2, training_dataloader=training_dataloader):
    net.to(device)

    for epoch in range(n_epochs):  # loop over the dataset multiple times
        for i, data in enumerate(training_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            #print(outputs)
            loss = criterion(outputs, labels)
            loss.backward()
                        # After the backward pass
            # for name, param in vidNet.named_parameters():
            #     if param.requires_grad and param.grad is not None:
            #         print(f'Parameter: {name}, Gradient norm: {param.grad.norm()}')
            optimizer.step()

            # print statistics
            if i % 10 == 0:
                print(f'Epoch={epoch + 1} Iter={i + 1:5d} Loss={loss.item():.3f}')
                running_loss = 0.0
    print('Finished Training')
    return net

C:\Users\jakes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
#train_network(vidNet, n_epochs=2)

In [8]:
#torch.save(vidNet.state_dict(), "pitchVidNetV1.pt")
#torch.save(vidNet.state_dict(), "pitchVidNetV2.pt")

In [9]:
# model = VideoConvNet()
# model.load_state_dict(torch.load("pitchVidNetV1.pt"))
# model = model.to('cuda')

In [10]:
# Test the model on the test dataset

def evaluate_model(model, test_dataloader):
    dataiter = iter(test_dataloader)

    running_loss = 0.0
    running_count = 0

    # Set model to evaluation mode
    model.eval()

    with torch.no_grad():
        for data, labels in dataiter:
            # Move data and labels to the device (GPU)
            data = data.to(device)
            labels = labels.to(device)

            # Forward pass
            predictions = model(data)

            # Compute loss
            loss = criterion(predictions, labels)  # Use appropriate loss function

            # Accumulate loss
            running_loss += loss.item()
            running_count += 1
            #print(f'Batch Loss: {loss.item():.6f}')

    # Calculate average loss
    average_loss = running_loss / running_count

    print(f'Average Loss: {average_loss:.6f}')

In [11]:
import random 

intensity = 'i'
harmonics = 'h'
jitter = 'j'
shimmer = 's'
pitch = 'p'

In [12]:



# Test the model on the test dataset

random.shuffle(video_paths)

training_intestity_paths = video_paths[:int(len(video_paths) * 0.8)]
test_intensity_paths = video_paths[int(len(video_paths) * 0.8):]

training_intensity_dataset = VideoDataset(training_intestity_paths, label_type=intensity)
test_intensity_dataset = VideoDataset(test_intensity_paths, label_type=intensity)

training_intensity_dataloader = DataLoader(training_intensity_dataset, batch_size=32, shuffle=True)
test_intensity_dataloader = DataLoader(test_intensity_dataset, batch_size=4, shuffle=True)

intensity_model = VideoConvNet()

train_network(intensity_model, n_epochs=2, training_dataloader=training_intensity_dataloader)

torch.save(intensity_model.state_dict(), "intensityVidNetV2.pt")

Epoch=1 Iter=    1 Loss=81.384
Epoch=1 Iter=   11 Loss=90.626
Epoch=1 Iter=   21 Loss=86.433
Epoch=1 Iter=   31 Loss=81.221
Epoch=1 Iter=   41 Loss=88.416
Epoch=1 Iter=   51 Loss=86.030
Epoch=1 Iter=   61 Loss=88.841
Epoch=1 Iter=   71 Loss=87.319
Epoch=1 Iter=   81 Loss=87.693
Epoch=1 Iter=   91 Loss=85.824
Epoch=1 Iter=  101 Loss=83.293
Epoch=1 Iter=  111 Loss=93.961
Epoch=1 Iter=  121 Loss=84.156
Epoch=1 Iter=  131 Loss=81.796
Epoch=1 Iter=  141 Loss=87.336
Epoch=1 Iter=  151 Loss=82.301
Epoch=1 Iter=  161 Loss=84.221
Epoch=1 Iter=  171 Loss=90.614
Epoch=1 Iter=  181 Loss=89.163
Epoch=1 Iter=  191 Loss=87.062
Epoch=1 Iter=  201 Loss=89.226
Epoch=1 Iter=  211 Loss=82.233
Epoch=1 Iter=  221 Loss=83.233
Epoch=1 Iter=  231 Loss=84.836
Epoch=1 Iter=  241 Loss=82.923
Epoch=1 Iter=  251 Loss=82.247
Epoch=1 Iter=  261 Loss=86.344
Epoch=1 Iter=  271 Loss=87.586
Epoch=1 Iter=  281 Loss=89.096
Epoch=1 Iter=  291 Loss=88.989
Epoch=1 Iter=  301 Loss=87.585
Epoch=1 Iter=  311 Loss=86.194
Epoch=1 

In [13]:
random.shuffle(video_paths)

training_pitch_paths = video_paths[:int(len(video_paths) * 0.8)]
test_pitch_paths = video_paths[int(len(video_paths) * 0.8):]

training_pitch_dataset = VideoDataset(training_pitch_paths, label_type=pitch)
test_pitch_dataset = VideoDataset(test_pitch_paths, label_type=pitch)

training_pitch_dataloader = DataLoader(training_pitch_dataset, batch_size=32, shuffle=True)
test_pitch_dataloader = DataLoader(test_pitch_dataset, batch_size=4, shuffle=True)

pitch_model = VideoConvNet()

train_network(pitch_model, n_epochs=2, training_dataloader=training_pitch_dataloader)

torch.save(pitch_model.state_dict(), "pitchVidNetV4.pt")

Epoch=1 Iter=    1 Loss=25.282
Epoch=1 Iter=   11 Loss=27.857
Epoch=1 Iter=   21 Loss=27.367
Epoch=1 Iter=   31 Loss=27.395
Epoch=1 Iter=   41 Loss=25.474
Epoch=1 Iter=   51 Loss=27.306
Epoch=1 Iter=   61 Loss=29.393
Epoch=1 Iter=   71 Loss=24.755
Epoch=1 Iter=   81 Loss=23.908
Epoch=1 Iter=   91 Loss=24.277
Epoch=1 Iter=  101 Loss=27.912
Epoch=1 Iter=  111 Loss=25.793
Epoch=1 Iter=  121 Loss=26.420
Epoch=1 Iter=  131 Loss=24.710
Epoch=1 Iter=  141 Loss=22.510
Epoch=1 Iter=  151 Loss=23.709
Epoch=1 Iter=  161 Loss=30.524
Epoch=1 Iter=  171 Loss=28.791
Epoch=1 Iter=  181 Loss=25.583
Epoch=1 Iter=  191 Loss=28.433
Epoch=1 Iter=  201 Loss=25.648
Epoch=1 Iter=  211 Loss=30.864
Epoch=1 Iter=  221 Loss=24.761
Epoch=1 Iter=  231 Loss=25.201
Epoch=1 Iter=  241 Loss=27.288
Epoch=1 Iter=  251 Loss=24.748
Epoch=1 Iter=  261 Loss=28.857
Epoch=1 Iter=  271 Loss=25.491
Epoch=1 Iter=  281 Loss=27.334
Epoch=1 Iter=  291 Loss=25.719
Epoch=1 Iter=  301 Loss=24.519
Epoch=1 Iter=  311 Loss=25.263
Epoch=1 

In [14]:
random.shuffle(video_paths)

training_harmonics_paths = video_paths[:int(len(video_paths) * 0.8)]
test_harmonics_paths = video_paths[int(len(video_paths) * 0.8):]

training_harmonics_dataset = VideoDataset(training_harmonics_paths, label_type=harmonics)
test_harmonics_dataset = VideoDataset(test_harmonics_paths, label_type=harmonics)

training_harmonics_dataloader = DataLoader(training_harmonics_dataset, batch_size=12, shuffle=True)
test_harmonics_dataloader = DataLoader(test_harmonics_dataset, batch_size=4, shuffle=True)

harmonics_model = VideoConvNet()

train_network(harmonics_model, n_epochs=2, training_dataloader=training_harmonics_dataloader)

torch.save(harmonics_model.state_dict(), "harmonicsVidNetV2.pt")

Epoch=1 Iter=    1 Loss=71.723
Epoch=1 Iter=   11 Loss=72.949
Epoch=1 Iter=   21 Loss=71.372
Epoch=1 Iter=   31 Loss=70.398
Epoch=1 Iter=   41 Loss=72.916
Epoch=1 Iter=   51 Loss=70.453
Epoch=1 Iter=   61 Loss=72.018
Epoch=1 Iter=   71 Loss=71.934
Epoch=1 Iter=   81 Loss=71.013
Epoch=1 Iter=   91 Loss=71.134
Epoch=1 Iter=  101 Loss=70.239
Epoch=1 Iter=  111 Loss=70.194
Epoch=1 Iter=  121 Loss=71.218
Epoch=1 Iter=  131 Loss=70.825
Epoch=1 Iter=  141 Loss=71.641
Epoch=1 Iter=  151 Loss=74.484
Epoch=1 Iter=  161 Loss=70.908
Epoch=1 Iter=  171 Loss=70.779
Epoch=1 Iter=  181 Loss=72.115
Epoch=1 Iter=  191 Loss=73.080
Epoch=1 Iter=  201 Loss=73.504
Epoch=1 Iter=  211 Loss=72.956
Epoch=1 Iter=  221 Loss=72.352
Epoch=1 Iter=  231 Loss=71.561
Epoch=1 Iter=  241 Loss=69.801
Epoch=1 Iter=  251 Loss=70.535
Epoch=1 Iter=  261 Loss=72.152
Epoch=1 Iter=  271 Loss=73.261
Epoch=1 Iter=  281 Loss=71.243
Epoch=1 Iter=  291 Loss=71.805
Epoch=1 Iter=  301 Loss=70.877
Epoch=1 Iter=  311 Loss=70.474
Epoch=1 

In [15]:
random.shuffle(video_paths)

training_jitter_paths = video_paths[:int(len(video_paths) * 0.8)]
test_jitter_paths = video_paths[int(len(video_paths) * 0.8):]

training_jitter_dataset = VideoDataset(training_jitter_paths, label_type=jitter)
test_jitter_dataset = VideoDataset(test_jitter_paths, label_type=jitter)

training_jitter_dataloader = DataLoader(training_jitter_dataset, batch_size=12, shuffle=True)
test_jitter_dataloader = DataLoader(test_jitter_dataset, batch_size=4, shuffle=True)

jitter_model = VideoConvNet()

train_network(jitter_model, n_epochs=2, training_dataloader=training_jitter_dataloader)

torch.save(jitter_model.state_dict(), "jitterVidNetV2.pt")

Epoch=1 Iter=    1 Loss=3.333
Epoch=1 Iter=   11 Loss=3.435
Epoch=1 Iter=   21 Loss=3.234
Epoch=1 Iter=   31 Loss=3.262
Epoch=1 Iter=   41 Loss=3.957
Epoch=1 Iter=   51 Loss=2.708
Epoch=1 Iter=   61 Loss=2.604
Epoch=1 Iter=   71 Loss=3.626
Epoch=1 Iter=   81 Loss=3.270
Epoch=1 Iter=   91 Loss=3.142
Epoch=1 Iter=  101 Loss=3.523
Epoch=1 Iter=  111 Loss=2.987
Epoch=1 Iter=  121 Loss=3.063
Epoch=1 Iter=  131 Loss=2.920
Epoch=1 Iter=  141 Loss=3.890
Epoch=1 Iter=  151 Loss=3.330
Epoch=1 Iter=  161 Loss=3.381
Epoch=1 Iter=  171 Loss=2.828
Epoch=1 Iter=  181 Loss=3.204
Epoch=1 Iter=  191 Loss=3.065
Epoch=1 Iter=  201 Loss=3.335
Epoch=1 Iter=  211 Loss=3.125
Epoch=1 Iter=  221 Loss=3.557
Epoch=1 Iter=  231 Loss=3.371
Epoch=1 Iter=  241 Loss=2.815
Epoch=1 Iter=  251 Loss=3.207
Epoch=1 Iter=  261 Loss=2.733
Epoch=1 Iter=  271 Loss=3.365
Epoch=1 Iter=  281 Loss=2.934
Epoch=1 Iter=  291 Loss=3.789
Epoch=1 Iter=  301 Loss=3.452
Epoch=1 Iter=  311 Loss=3.302
Epoch=1 Iter=  321 Loss=3.395
Epoch=1 It

In [16]:
random.shuffle(video_paths)

training_shimmer_paths = video_paths[:int(len(video_paths) * 0.8)]
test_shimmer_paths = video_paths[int(len(video_paths) * 0.8):]

training_shimmer_dataset = VideoDataset(training_shimmer_paths, label_type=shimmer)
test_shimmer_dataset = VideoDataset(test_shimmer_paths, label_type=shimmer)

training_shimmer_dataloader = DataLoader(training_shimmer_dataset, batch_size=12, shuffle=True)
test_shimmer_dataloader = DataLoader(test_shimmer_dataset, batch_size=4, shuffle=True)

shimmer_model = VideoConvNet()

train_network(shimmer_model, n_epochs=2, training_dataloader=training_shimmer_dataloader)

torch.save(shimmer_model.state_dict(), "shimmerVidNetV2.pt")

Epoch=1 Iter=    1 Loss=7.487
Epoch=1 Iter=   11 Loss=5.959
Epoch=1 Iter=   21 Loss=6.468
Epoch=1 Iter=   31 Loss=6.896
Epoch=1 Iter=   41 Loss=7.129
Epoch=1 Iter=   51 Loss=5.869
Epoch=1 Iter=   61 Loss=8.001
Epoch=1 Iter=   71 Loss=5.817
Epoch=1 Iter=   81 Loss=6.486
Epoch=1 Iter=   91 Loss=5.999
Epoch=1 Iter=  101 Loss=5.907
Epoch=1 Iter=  111 Loss=5.183
Epoch=1 Iter=  121 Loss=7.335
Epoch=1 Iter=  131 Loss=6.237
Epoch=1 Iter=  141 Loss=6.219
Epoch=1 Iter=  151 Loss=7.078
Epoch=1 Iter=  161 Loss=7.041
Epoch=1 Iter=  171 Loss=6.894
Epoch=1 Iter=  181 Loss=7.101
Epoch=1 Iter=  191 Loss=5.478
Epoch=1 Iter=  201 Loss=6.689
Epoch=1 Iter=  211 Loss=7.142
Epoch=1 Iter=  221 Loss=5.882
Epoch=1 Iter=  231 Loss=6.472
Epoch=1 Iter=  241 Loss=7.323
Epoch=1 Iter=  251 Loss=5.879
Epoch=1 Iter=  261 Loss=6.525
Epoch=1 Iter=  271 Loss=7.232
Epoch=1 Iter=  281 Loss=6.756
Epoch=1 Iter=  291 Loss=6.712
Epoch=1 Iter=  301 Loss=7.500
Epoch=1 Iter=  311 Loss=6.387
Epoch=1 Iter=  321 Loss=6.076
Epoch=1 It

In [17]:
#print rows of pickle

video_path = ENV_FOLDER_DATA_PROC + 'clips\\595\\595_seg1434.mp4'
#video_path = ENV_FOLDER_DATA_PROC + 'clips\\542\\542_seg1963.mp4'

label_type = 'p'    
episode_number = video_path.split('\\')[-2]
seg_number = video_path.split('_seg')[-1].split('.')[0]
pickle_path = ENV_FOLDER_DATA_PROC + 'targetdf\\' + episode_number + '.pickle'
target_df = pd.read_pickle(pickle_path)

#find row where seg number is equal to seg number and type is equal to label type
#print(target_df)
target_row = target_df[(target_df['seg'] == int(seg_number)) & (target_df['type'] == label_type)]
#print(target_row)

#extract the data from columns 3:43

audio_data = target_row.iloc[:, 4:].values


# print(episode_number,seg_number, pickle_path, label_type)
print(audio_data)


#replace all nan values with 0


labels = torch.tensor(audio_data.astype(float).flatten())

# print(audio_data.shape)
# print(audio_data.flatten())
# print(audio_data.flatten().shape)

labels = torch.nan_to_num(labels)

print(labels)



[[152.71588313          nan 115.52315062 127.2030195  136.41863782
  158.54772876 166.05814081          nan 189.93624618 172.78173102
           nan 180.74059225 169.08045221          nan 132.92932199
  153.36558848 137.24713367 154.65423365 145.86324941 125.35254113
  138.12565524 155.73557724 154.30628446 387.19012768 140.55706241
  148.83682277 127.93065484 131.54946958 152.3069015  160.59326155
  165.89942255 119.68279223          nan 162.69793442 151.89071249
  132.67762781 101.38566823          nan          nan 173.13215875
           nan]]
tensor([152.7159,   0.0000, 115.5232, 127.2030, 136.4186, 158.5477, 166.0581,
          0.0000, 189.9362, 172.7817,   0.0000, 180.7406, 169.0805,   0.0000,
        132.9293, 153.3656, 137.2471, 154.6542, 145.8632, 125.3525, 138.1257,
        155.7356, 154.3063, 387.1901, 140.5571, 148.8368, 127.9307, 131.5495,
        152.3069, 160.5933, 165.8994, 119.6828,   0.0000, 162.6979, 151.8907,
        132.6776, 101.3857,   0.0000,   0.0000, 173.1322,